## Using apply_ufunc

http://xarray.pydata.org/en/stable/generated/xarray.apply_ufunc.html

http://xarray.pydata.org/en/stable/examples/apply_ufunc_vectorize_1d.html

We have had a couple of instances where we had functions that only worked on `numpy` arrays and not on `xarray.Datasets` or `xarray.DataArrays`

For example, when using `scipy` and `numpy` functions.  

In these cases, we had to loop over our data in space and apply the function to each point. 

`apply_ufunc` applies functions to `xarray.DataArrays` and handles the looping more efficiently than we would directly. 

In [1]:
import xarray as xr
import numpy as np
from scipy.stats import linregress

In [65]:
def lreg(x,y):

    '''
    x_ts is our timeseries (e.g. nino34)
    y_maps is our maps in time (e.g. precip anomalies)
    '''

    print(x.shape,y.shape)
    
    m = linregress(x,y)
    
    return m

In [66]:
file_nino34='/scratch/kpegion/nino34_1982-2019.oisstv2_anoms.nc'
ds_nino34=xr.open_dataset(file_nino34)

In [67]:
file='/shared/obs/gridded/GPCP/monthly/precip.mon.mean.nc'
ds_precip=xr.open_dataset(file)

ds_precip=ds_precip.sel(time=slice(ds_nino34['time'][0],ds_nino34['time'][-1]))

ds_climo=ds_precip.groupby('time.month').mean()
ds_anoms=ds_precip.groupby('time.month')-ds_climo

In [68]:
tmp=lreg(ds_nino34['sst'],
         ds_anoms['precip'].isel(lat=0,lon=0))
print(tmp)


(456,) (456,)
LinregressResult(slope=-0.014856616498533752, intercept=-1.4356906681244804e-08, rvalue=-0.05694430633070501, pvalue=0.22488329211272526, stderr=0.012224649103584444)


In [69]:
xr.apply_ufunc(lreg,ds_nino34['sst'],
               ds_anoms['precip'],
               input_core_dims=[['time'],['time','lat','lon']],
               output_core_dims=['lat','lon'],
               vectorize=True)

(456,) (456, 72, 144)


ValueError: y has more than 2 dimensions